**ASSIGNMENT-7**

In [ ]:
!pip install torch torchvision transformers pillow clip-by-openai


INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://p

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import requests
from io import BytesIO


In [ ]:
# Load the CLIP processor and model
model_name = "openai/clip-vit-base-patch32"  # You can choose other variants
processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name)


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
# Example image from a URL (you can also use a local image file)
image_url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Preprocess the image using the processor
inputs = processor(images=img, return_tensors="pt", padding=True)


In [ ]:
# Possible captions for the image
captions = [
    "A dog sitting on the grass.",
    "A beautiful sunset over the ocean.",
    "A person riding a bike on the street.",
    "A red sports car parked on the street.",
    "A cat sitting in front of a laptop."
]

# Preprocess the texts (captions)
text_inputs = processor(text=captions, return_tensors="pt", padding=True)


In [ ]:
# Get the image and text embeddings from the model
with torch.no_grad():
    image_features = model.get_image_features(**inputs)
    text_features = model.get_text_features(**text_inputs)

# Normalize the features (vectors) to unit length for cosine similarity
image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)


In [ ]:
# Compute cosine similarity between the image and the captions
cosine_similarities = torch.matmul(image_features, text_features.T)

# Get the index of the caption with the highest similarity
best_caption_idx = torch.argmax(cosine_similarities, dim=1).item()

# Display the best caption
best_caption = captions[best_caption_idx]
print("Generated Caption:", best_caption)


Generated Caption: A dog sitting on the grass.
